In [6]:
from pprint import pprint 
from elasticsearch import Elasticsearch 

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'tdRTedevS6WZDfd1oEOoeQ',
 'name': 'd3f17afee3cf',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-01-10T10:08:26.972230187Z',
             'build_flavor': 'default',
             'build_hash': 'd4b391d925c31d262eb767b8b2db8f398103f909',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.12.0',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.17.1'}}


In [7]:
es.indices.delete(index='products', ignore_unavailable=True)
es.indices.create(index='products')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'})

In [8]:
import json 

dummy_data = json.load(open("data/dummy_data_2.json"))
for _ in range(10):
    dummy_data += dummy_data

len(dummy_data)

5120

In [11]:
# using bulk operations 

operations = []
for document in dummy_data:
    operations.append({'index': {'_index': 'products'}})
    operations.append(document)

es.bulk(operations=operations)

ObjectApiResponse({'errors': False, 'took': 801, 'items': [{'index': {'_index': 'products', '_id': 'DUXp-JQBqAMUwyyAwhVz', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'products', '_id': 'DkXp-JQBqAMUwyyAwhVz', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'products', '_id': 'D0Xp-JQBqAMUwyyAwhVz', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'products', '_id': 'EEXp-JQBqAMUwyyAwhVz', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'products', '_id': 'EUXp-JQBqAMUwyyAwhVz', '_version': 1, 'result': 'created', '_shards': {'tota

In [12]:
# size and from 

response = es.search(
    index='products',
    body={
        "query": {
            "match_all": {}
        },
        "size": 10,
        "from": 10
    },
)

for hit in response['hits']['hits']:
    print(hit['_source'])

{'message': 'This is an important keyword search result.', 'age': 25, 'price': 100.0}
{'message': 'Another search result with an important keyword.', 'age': 30, 'price': 150.0}
{'message': 'Keyword match in this result as well.', 'age': 40, 'price': 200.0}
{'message': 'Important keyword again in this document.', 'age': 35, 'price': 120.0}
{'message': 'Final document with the important keyword.', 'age': 28, 'price': 180.0}
{'message': 'This is an important keyword search result.', 'age': 25, 'price': 100.0}
{'message': 'Another search result with an important keyword.', 'age': 30, 'price': 150.0}
{'message': 'Keyword match in this result as well.', 'age': 40, 'price': 200.0}
{'message': 'Important keyword again in this document.', 'age': 35, 'price': 120.0}
{'message': 'Final document with the important keyword.', 'age': 28, 'price': 180.0}


In [27]:
# timeout 

response = es.search(
    index='products',
    body={
        "query": {
            "match": {
                "message": "search keyword"
            }
        },
        "timeout": "1s"
    },
)

response.body

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5120, 'relation': 'eq'},
  'max_score': 0.8941701,
  'hits': [{'_index': 'products',
    '_id': 'DUXp-JQBqAMUwyyAwhVz',
    '_score': 0.8941701,
    '_source': {'message': 'This is an important keyword search result.',
     'age': 25,
     'price': 100.0}},
   {'_index': 'products',
    '_id': 'DkXp-JQBqAMUwyyAwhVz',
    '_score': 0.8941701,
    '_source': {'message': 'Another search result with an important keyword.',
     'age': 30,
     'price': 150.0}},
   {'_index': 'products',
    '_id': 'EkXp-JQBqAMUwyyAwhVz',
    '_score': 0.8941701,
    '_source': {'message': 'This is an important keyword search result.',
     'age': 25,
     'price': 100.0}},
   {'_index': 'products',
    '_id': 'E0Xp-JQBqAMUwyyAwhVz',
    '_score': 0.8941701,
    '_source': {'message': 'Another search result with an important keyword.',
     'age': 30,
     'price': 150.0}},
   

In [29]:
# aggregation 

response = es.search(
    index='products',
    body={
        "query": {
            "match_all": {}
        },
        "aggs": {
            "avg_age":{
                "avg":{
                    "field": "age"
                }
            }
        }
    }
)

average_age = response['aggregations']['avg_age']['value']
print(f"Average Age: {average_age}")

Average Age: 31.6


In [33]:
# all combined 

response = es.search(
    index='products',
    body={
        "query": {
            "match": {
                "message": "important keyword"
            }
        },
        "aggs": {
            "max_price":{
                "max":{
                    "field": "price"
                }
            }
        },
        "size": 50,
        "from": 50,
        "timeout": "2s"
    }
)

for hit in response['hits']['hits']:
    print(hit['_source'])

max_price = response['aggregations']['max_price']['value']
print(f"Max Price: {max_price}")

{'message': 'Important keyword again in this document.', 'age': 35, 'price': 120.0}
{'message': 'Final document with the important keyword.', 'age': 28, 'price': 180.0}
{'message': 'Important keyword again in this document.', 'age': 35, 'price': 120.0}
{'message': 'Final document with the important keyword.', 'age': 28, 'price': 180.0}
{'message': 'Important keyword again in this document.', 'age': 35, 'price': 120.0}
{'message': 'Final document with the important keyword.', 'age': 28, 'price': 180.0}
{'message': 'Important keyword again in this document.', 'age': 35, 'price': 120.0}
{'message': 'Final document with the important keyword.', 'age': 28, 'price': 180.0}
{'message': 'Important keyword again in this document.', 'age': 35, 'price': 120.0}
{'message': 'Final document with the important keyword.', 'age': 28, 'price': 180.0}
{'message': 'Important keyword again in this document.', 'age': 35, 'price': 120.0}
{'message': 'Final document with the important keyword.', 'age': 28, 'p